In [46]:
import sys
# sys.path.append('J:\\Quant\\freq_used_funcs')
print(sys.path)
# 获取tushare每股营业收入数据
import tushare as ts
import pandas as pd
import datetime
import numpy as np
pro = ts.pro_api('e9628663b45f87ea92e39aaa7127063830ebd5090bc1e8943138b84f')
import security_basic_info

['D:\\program\\PyCharm 2022.3.3\\plugins\\python\\helpers-pro\\jupyter_debug', 'D:\\program\\PyCharm 2022.3.3\\plugins\\python\\helpers\\pydev', 'J:\\Quant\\JQ\\scap_growth', 'J:\\Quant\\JQ\\scap_growth', 'D:\\program\\python310.zip', 'D:\\program\\DLLs', 'D:\\program\\lib', 'D:\\program', 'J:\\Quant\\junhe_quant\\scap_growth', '', 'J:\\Quant\\junhe_quant\\scap_growth\\lib\\site-packages', 'J:\\Quant\\junhe_quant\\scap_growth\\lib\\site-packages\\win32', 'J:\\Quant\\junhe_quant\\scap_growth\\lib\\site-packages\\win32\\lib', 'J:\\Quant\\junhe_quant\\scap_growth\\lib\\site-packages\\Pythonwin', 'J:\\Quant\\freq_used_funcs', 'J:\\Quant\\freq_used_funcs', 'J:\\Quant\\freq_used_funcs']


In [20]:
start_date = '2005-01-01'
# 建立交易日历

today = datetime.date.today().strftime('%Y-%m-%d')
df = pro.trade_cal()
df
# trade_date_all

,exchange,cal_date,is_open,pretrade_date
0,SSE,20231231,0,20231229
1,SSE,20231230,0,20231229
2,SSE,20231229,1,20231228
3,SSE,20231228,1,20231227
4,SSE,20231227,1,20231226
...,...,...,...,...
12061,SSE,19901223,0,19901221
12062,SSE,19901222,0,19901221
12063,SSE,19901221,1,19901220
12064,SSE,19901220,1,19901219


In [21]:
trade_date_all = df.loc[df['is_open']==1,'cal_date'].tolist()

In [31]:
trade_date_all.sort()

In [32]:
years = 15
# 获取当前日期
today = datetime.date.today()
# 计算n年前的日期
ten_years_ago = today - datetime.timedelta(days=365*years)

# 生成月份列表
months = []
year_month = lambda x: x.strftime('%Y%m')

for year in range(ten_years_ago.year, today.year+1):
    for month in range(1, 13):
        date = datetime.date(year, month, 1)
        if ten_years_ago <= date <= today:
            months.append(year_month(date))

# print(months)

In [33]:
# 获取每月第一个交易日
first_trade_date_each_month = []
for i in months[:-3]:
    first_trade_date_each_month.append([x for x in trade_date_all if i in x][0])
# first_trade_date_each_month

In [64]:
date = first_trade_date_each_month[-20]

In [70]:
df = pro.daily_basic(ts_code='', trade_date=date, fields=['ts_code','trade_date','turnover_rate','volume_ratio','pe','pb','circ_mv'])

In [72]:
date

'20210506'

In [66]:
factor_data =df.dropna()
factor_data = factor_data.sort_values(by='circ_mv', ascending=True)

In [67]:
# 将数据分成10段，并计算每段的长度
n = 10
step = len(factor_data) // n
ranges = [(i * step, (i+1) * step - 1) for i in range(n-1)]
ranges.append(((n-1) * step, len(factor_data)-1))
# 将数据分成10个list，并分别存储column2的值
sec_by_group = []
for r in ranges:
    sec_by_group.append(list(factor_data.iloc[r[0]:r[1]+1]['ts_code']))

In [68]:
# 获取证券此后20天的收益率
# 获取此后第20个交易日的日期
i=0
date = first_trade_date_each_month[-20]
average_return_by_group = []
for i in range(len(sec_by_group)):
    date_20days_after = [x for x in trade_date_all if x>date][20]
    return_ls =[]
    for sec in sec_by_group[i]:
        single_df = ts.pro_bar(ts_code=sec, adj='qfq', start_date=date, end_date=date_20days_after)
    ret = single_df.close.iloc[-1]/single_df.close.iloc[0]-1
    return_ls.append(ret)
    average_return = np.nanmean(return_ls)
    average_return_by_group.append(average_return)
average_return_by_group

KeyboardInterrupt: 

In [45]:
#取000001的前复权行情
df = ts.pro_bar(ts_code='000001.SZ', adj='qfq', start_date='20180101', end_date='20181011')

In [61]:
ts.pro_bar(ts_code=sec, adj='qfq', start_date=date, end_date=date_20days_after)

In [69]:
df = pro.monthly(trade_date='20181031', fields='ts_code,trade_date,change')